# Example 04 2.5 duplex

In [1]:
from LoopStructural import GeologicalModel
from LoopStructural.visualisation.model_visualisation import LavaVuModelViewer
import numpy as np
import pandas as pd

In [2]:
class RegionFeature:
    def __init__(self, function):
        self.function = function
        self.name = 'region'
    def evaluate_value(self, pos):
        return self.function(pos).astype(float)
    def mean(self):
        return 0
    def max(self):
        return 1
    def min(self):
        return -1
    def name(self):
        return 'region'

In [3]:
boundary_points = np.zeros((2,3))
boundary_points[0,0] = -40
boundary_points[0,1] = -5
boundary_points[0,2] = -10
boundary_points[1,0] = 40
boundary_points[1,1] = 5
boundary_points[1,2] = 10

In [4]:
data = pd.read_pickle('duplex.pkl')
data = data.rename(columns = {"V":"val"})
data.head()

,X,Y,Z,val,type,nx,ny,nz,coord
0,6.10,0.1,1.1,0.0,s0,NaN,NaN,NaN,NaN
1,6.10,0.1,2.1,1.0,s0,NaN,NaN,NaN,NaN
0,1.00,1.0,1.0,NaN,s0,0.0,-0.0,1.0,NaN
0,-18.17,-5.0,-4.0,0.0,fault1,NaN,NaN,NaN,0.0
1,-15.17,-5.0,-4.0,0.0,fault1,NaN,NaN,NaN,0.0


In [5]:
data[data['type'] == 's0']

,X,Y,Z,val,type,nx,ny,nz,coord
0,6.1,0.1,1.1,0.0,s0,NaN,NaN,NaN,NaN
1,6.1,0.1,2.1,1.0,s0,NaN,NaN,NaN,NaN
0,1.0,1.0,1.0,NaN,s0,0.0,-0.0,1.0,NaN


In [6]:
from scipy.sparse import diags 
def precon(A):
    return diags(1 / A.diagonal(),0)
interpolation_region = lambda pos : pos[:,0] > 0.1

In [7]:
model = GeologicalModel(boundary_points[0,:],boundary_points[1,:])
model.set_model_data(data)
fault1 = model.create_and_add_fault('fault1',
                                    -4.5,
                                    nelements=5000,
                                    solver='lu',
                                    interpolatortype='PLI',
#                                    damp = True,
                                    precon=precon,
                                    buffer=0.1,
#                                                                         region=interpolation_region

                                   )


1574
1574
1574


In [8]:
# splayregion = lambda pos : ~np.logical_and(fault1['feature'][1].evaluate_value(pos)>-0.25,
#                                            fault1['feature'][1].evaluate_value(pos)<1.5)
# interpolation_region = lambda pos : np.logical_and(fault1['feature'][1].evaluate_value(pos)>-0.3,
#                                            fault1['feature'][1].evaluate_value(pos)<1.55)
splayregion = lambda pos : pos[:,0] < 0.05
#np.logical_and(fault1['feature'][1].evaluate_value(pos)>-0.3,
                                           #fault1['feature'][1].evaluate_value(pos)<1.55)

In [9]:
fault2 = model.create_and_add_fault('fault2',
                                    -4.5,
                                    nelements=5000,
#                                     splayregion=splayregion,
#                                     splay=fault1['feature'],
                                   solver='lu',
                                   precon=precon,
                                   interpolatortype='PLI',
#                                    damp=True,
                                    buffer=0.1,
#                                     cpw=10,
                                    region=interpolation_region
                                   )
# # s0 = model.create_and_add_conformable_foliation('s0',
#                                                 nelements=2000,
#                                                 solver='lu',
#                                                 interpolatortype='FDI',
#                                                 precon=precon,
#                                                 region=interpolation_region,
#                                                        damp = True,
#                                                 buffer=0.1
#                                                )
# print(s0.get_interpolator().support.n_nodes)
# s02 = model.create_and_add_conformable_foliation('s0',
#                                                 nelements=2000,
#                                                 solver='lu',
#                                                 interpolatortype='FDI',
#                                                 precon=precon,
# #                                                 region=interpolation_region,
#                                                        damp = True,
#                                                 buffer=0.1
#                                                )
# print(s02.get_interpolator().support.n_nodes)


region kwarg
region kwarg
region kwarg
1256


ValueError: column index exceeds matrix dimensions

In [ ]:
# splay = RegionFeature(splayregion)
interp = RegionFeature(interpolation_region)

In [ ]:
#%%snakeviz
viewer = LavaVuModelViewer()
viewer.add_isosurface(fault1['feature'],
                      isovalue=0,
#                       voxet=model.voxet()
                     )

# # # # viewer.add_isosurface(fault1[1],
# # # #                       slices=[-0.25,.9],#nslices=5,
# # # #                       voxet=model.voxet())
viewer.add_isosurface(fault2['feature'],
                      isovalue=0,
#                       voxet=model.voxet()
                     )
# # viewer.add_scalar_field(model.bounding_box,(38,55,30),
# #                       'box',
# #                      paint_with=splay,
# #                      cmap='prism')
viewer.add_scalar_field(model.bounding_box,(38,55,30),
                      'box2',
                     paint_with=interp,
                     cmap='prism')
# viewer.add_scalar_field(model.bounding_box,(38,55,30),
#                       'box3',
#                      paint_with=s0,
#                      cmap='prism')
# # # viewer.add_scalar_field(model.bounding_box,(38,55,30),
# # #                       'box2',
# # #                      paint_with=s0,
# # #                      cmap='prism')
# viewer.add_scalar_field(model.bounding_box,(38,55,30),
#                       'box3',
#                      paint_with=fault1['feature'],
#                      cmap='prism')
# viewer.add_isosurface(s0,
#                       isovalue=0,
# #                       nslices=10,
#                      voxet=model.voxet()
#                      )
# viewer.add_isosurface(s02,
#                       name='test',
#                       isovalue=0,
# #                       nslices=10,
#                      voxet=model.voxet()
#                      )
# viewer.add_section()
# # viewer.add_isosurface(splay,voxet=model.voxet())
# # viewer.add_data(fault2[0])
# viewer.add_data(fault1['feature'][0])
# # viewer.add_data(fault2['feature'][0])
# viewer.add_points(s0.get_interpolator().support.nodes,name='points')
# viewer.add_points(s02.get_interpolator().support.nodes,name='points2')

# viewer.add_data(s0)

viewer.interactive()